In [1]:
import os
import sys
print(f"{sys.executable = }")
print(f"{os.getcwd() = }")
sys.path.append('/nas/home/sh0416/sentence-benchmark/src')


sys.executable = '/home/sh0416/anaconda3/envs/sentence-benchmark/bin/python'
os.getcwd() = '/nas/home/sh0416/sentence-benchmark/src/notebooks'


In [2]:
root = "/nas/home/sh0416/data/semeval16/task2/test_goldStandard"
# For bert-base-uncased
model_args_path = "/home/sh0416/checkpoints/20211021_113441/model_args.json"
checkpoint_path = "/home/sh0416/checkpoints/20211021_113441/checkpoint-1500/pytorch_model.bin"
# For roberta-base
# model_args_path = "/home/sh0416/checkpoints/20211021_152037/model_args.json"
# checkpoint_path = "/home/sh0416/checkpoints/20211021_152037/checkpoint-2000/pytorch_model.bin"
sources = ["images", "headlines", "answers-students"]

filepaths_sent1 = [f"STSint.testinput.{source}.sent1.txt" for source in sources]
filepaths_sent2 = [f"STSint.testinput.{source}.sent2.txt" for source in sources]
filepaths_sent1_tok = [f"STSint.testinput.{source}.sent1.chunk.txt" for source in sources]
filepaths_sent2_tok = [f"STSint.testinput.{source}.sent2.chunk.txt" for source in sources]
filepaths_align = [f"STSint.testinput.{source}.wa" for source in sources]

In [3]:
import os
from simcse.eval.ists import load_alignment
from simcse.eval.ists import save_alignment
data = load_alignment(os.path.join(root, filepaths_align[0]))
save_alignment(data, "test.wa")

In [4]:
from simcse.eval.ists import load_instances
instances = load_instances(
    os.path.join(root, filepaths_sent1[1]),
    os.path.join(root, filepaths_sent2[1]),
    os.path.join(root, filepaths_sent1_tok[1]),
    os.path.join(root, filepaths_sent2_tok[1])
)
instances[0]

{'id': 1,
 'sent1': "China 's Peace Ark departs for the Philippines Thursday",
 'sent2': 'China sends aid team to Philippines',
 'sent1_chunk': ['China',
  "'s Peace Ark",
  'departs',
  'for the Philippines',
  'Thursday'],
 'sent2_chunk': ['China', 'sends', 'aid team', 'to Philippines']}

In [5]:
import json
from transformers import AutoTokenizer
from simcse.config import ModelArguments

with open(model_args_path) as f:
    model_args = ModelArguments(**json.load(f))
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, use_fast=False)

In [6]:
model_args

ModelArguments(model_name_or_path='bert-base-uncased', temp=0.05, pooler_type='cls', hidden_dropout_prob=0.1, mlp_only_train=True, coeff_mlm=0.1, loss_rwmd=True, layer_idx=12)

In [7]:
from simcse.eval.ists import preprocess_instances

prep_instances = preprocess_instances(tokenizer, instances)
prep_instances[0]

{'instance': {'id': 1,
  'sent1': "China 's Peace Ark departs for the Philippines Thursday",
  'sent2': 'China sends aid team to Philippines',
  'sent1_chunk': ['China',
   "'s Peace Ark",
   'departs',
   'for the Philippines',
   'Thursday'],
  'sent2_chunk': ['China', 'sends', 'aid team', 'to Philippines']},
 'sent1_token': ['china',
  "'",
  's',
  'peace',
  'ark',
  'depart',
  '##s',
  'for',
  'the',
  'philippines',
  'thursday'],
 'sent2_token': ['china', 'sends', 'aid', 'team', 'to', 'philippines'],
 'inputs1': {'input_ids': tensor([[  101,  2859,  1005,  1055,  3521, 15745, 18280,  2015,  2005,  1996,
           5137,  9432,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 'inputs2': {'input_ids': tensor([[  101,  2859, 10255,  4681,  2136,  2000,  5137,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}}

In [8]:
import torch
from simcse.models import create_contrastive_learning

module = create_contrastive_learning(model_args)
module.load_state_dict(torch.load(checkpoint_path))
module.model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 31.74 GiB total capacity; 416.47 MiB already allocated; 5.69 MiB free; 452.00 MiB reserved in total by PyTorch)

In [17]:
from simcse.eval.ists import inference

infered_instances = inference(module.model, prep_instances)

In [18]:
infered_instances[0]["pairs"]

[{'sent1_word_ids': [1],
  'sent2_word_ids': [1],
  'type': 'EQUI',
  'score': 5,
  'comment': ''},
 {'sent1_word_ids': [6, 7, 8],
  'sent2_word_ids': [5, 6],
  'type': 'EQUI',
  'score': 5,
  'comment': ''}]

In [19]:
from simcse.eval.ists import save_infered_instances

save_infered_instances(infered_instances, f"{sources[1]}_{model_args.model_name_or_path}_rwmd-pool.wa")

In [ ]:
x = np.zeros((4, 2))
x[[0, 1], [0, 1]]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_heatmap(sentence):
    source = ['0 NIL : '] + sentence["source"]
    translation = ['0 NIL : '] + sentence["translation"]
    heatmap = np.zeros((len(source), len(translation)))
    for a in sentence["alignment"]:
        if a["score"] != 'NIL':
            token_id_seq1, token_id_seq2 = a["pair_id"]
            score = float(a["score"])
            for token_id1 in token_id_seq1:
                for token_id2 in token_id_seq2:
                    heatmap[token_id1, token_id2] = score

    fig, ax = plt.subplots(facecolor='white')
    ax.set_ylabel("source")
    ax.set_yticks(np.arange(0.5, len(source)+0.5, 1))
    ax.set_yticklabels(source)
    ax.set_xlabel("translation")
    ax.set_xticks(np.arange(0.5, len(translation)+0.5, 1))
    ax.set_xticklabels(translation, rotation=90)
    ax.pcolormesh(heatmap, cmap=plt.get_cmap('Blues'))
    ax.invert_yaxis()
    for a in sentence["alignment"]:
        if a["type"] != 'NOALI':
            token_id_seq1, token_id_seq2 = a["pair_id"]
            for token_id1 in token_id_seq1:
                for token_id2 in token_id_seq2:
                    ax.text(token_id2, token_id1+0.5, a["type"], c='w')
    plt.tight_layout()
    return fig, ax

for sentence in data[:100]:
    plot_heatmap(sentence)
    plt.show()

In [ ]:
from simcse.models import create_contrastive_learning
from simcse.config import ModelArguments

model_args = ModelArguments(loss_rwmd=True)
module = create_contrastive_learning(model_args)